In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
import ast
import warnings
warnings.filterwarnings("ignore")
import plotly.io as pio

In [2]:
# Set the path to your dataset folder
data_path = "../data/"

# Load each CSV into a DataFrame
df_places = pd.read_csv(data_path + "places.csv")
df_users = pd.read_csv(data_path + "users.csv")
df_reviews = pd.read_csv(data_path + "reviews.csv")

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


### Trying to clean the data to get only restaurants then use the google maps api on the rest to get even more information

#### Filter all businesses to only restaurants

In [18]:
df_places = df_places[df_places['price'].notna()]
display(df_places)

,name,price,phone,closed,gPlusPlaceId,address_line1,address_line2,address_line3,latitude,longitude,hours_monday,hours_tuesday,hours_wednesday,hours_thursday,hours_friday,hours_saturday,hours_sunday
3,T C's Referee Sports Bar,$$,(605) 361-2208,False,100327153115986850675,5322 W 26th St,"Sioux Falls, SD 57106",NaN,4.352949e+01,-9.679224e+01,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--10:00 pm
9,Old Chicago,$$,(503) 533-4650,False,118222137795476771294,17960 NW Evergreen Pkwy,"Beaverton, OR 97006",NaN,4.553518e+01,-1.228622e+02,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am
10,China Cottage,$$,(937) 294-4724,False,106432060150136868000,3718 Wilmington Pike,"Dayton, OH 45429",NaN,3.969290e+01,-8.413617e+01,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--11:00 pm,11:00 am--11:00 pm,11:00 am--10:00 pm
23,Smokey Mountain Wings,$$,(865) 465-3121,False,100184392614713668281,3607 Outdoor Sportsman Pl,"Kodak, TN 37764",NaN,3.598598e+01,-8.361060e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,Sabatinos Italian Kitchen,$$$,(781) 646-4126,False,110300304875024740707,242 Massachusetts Ave,"Arlington, MA 02474",NaN,4.240690e+01,-7.114399e+01,10:00 am--10:15 pm,10:00 am--10:15 pm,10:00 am--10:15 pm,10:00 am--10:15 pm,10:00 am--10:30 pm,10:00 am--10:30 pm,10:00 am--9:45 pm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114315,Coast Coffee Cafe,$$$,(08) 9481 8868,False,102445991423518108977,1269 Hay St,West Perth WA 6005,Australia,-3.194821e+01,1.158403e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3114320,Alvernon Donut Shop,$$$,(520) 326-3307,False,105165247399326889634,1450 S Alvernon Way,"Tucson, AZ 85711",NaN,3.220395e+01,-1.109099e+02,5:00 am--5:00 pm,5:00 am--5:00 pm,5:00 am--5:00 pm,5:00 am--5:00 pm,5:00 am--5:00 pm,5:00 am--5:00 pm,5:00 am--5:00 pm
3114323,SUBWAY® Restaurants,$$$,(906) 265-7827,False,102934499497015360471,Riverside Ave,"Iron River, MI 49935",NaN,4.608880e+07,-8.863522e+07,8:00 am--9:00 pm,8:00 am--9:00 pm,8:00 am--9:00 pm,8:00 am--9:00 pm,8:00 am--9:00 pm,8:00 am--8:00 pm,9:00 am--8:00 pm
3114324,Super Stop & Shop,$$,(860) 404-0341,False,109070835041469072310,1799 Farmington Ave,"Unionville, CT 06085",NaN,4.175697e+01,-7.288338e+01,7:00 am--12:00 am,7:00 am--12:00 am,7:00 am--12:00 am,7:00 am--12:00 am,7:00 am--12:00 am,7:00 am--12:00 am,7:00 am--10:00 pm


In [14]:
df_reviews = df_reviews[df_reviews['reviewText'].notna()]
df_reviews = df_reviews[df_reviews['categories'].notna()]
unique_values = df_reviews['categories'].unique()
restaurants = df_reviews[df_reviews['categories'].str.contains('restaurant', case=False)]

In [19]:
display(restaurants)

,rating,reviewerName,reviewText,categories,reviewTime,unixReviewTime,formattedDate,gPlusPlaceId,gPlusUserId
13,4.0,william spindler,Best War Wanton soup in Red Bluff,Asian Restaurant; Chinese Restaurant,"Mar 12, 2014",1.394669e+09,2014-03-13 01:11:36,106591714648856494903,100000032416892623125
14,5.0,william spindler,This is a review that is long overdo. I've bee...,European Restaurant; Italian Restaurant; Pizza...,"Mar 14, 2014",1.394826e+09,2014-03-14 20:46:28,109420033090810328045,100000032416892623125
15,5.0,william spindler,"Some authentic rub BBQ, great food and don't m...",Barbecue Restaurant,"Mar 12, 2014",1.394671e+09,2014-03-13 01:40:15,111623070919810985923,100000032416892623125
16,4.0,william spindler,Truly a Red Bluff standard. Great old fashione...,Restaurant,"Mar 12, 2014",1.394670e+09,2014-03-13 01:25:57,113854191152597312098,100000032416892623125
17,5.0,william spindler,"Long time favorite Mexican food, always consis...",Mexican Restaurant,"Mar 12, 2014",1.394670e+09,2014-03-13 01:15:13,115827996910815192564,100000032416892623125
...,...,...,...,...,...,...,...,...,...
11453812,2.0,charles mckinney,Looks and tastes like microwave pizza.,Delivery Restaurant,"Mar 30, 2013",1.364706e+09,2013-03-31 06:59:40,113241203971701227914,118446742455312620560
11453821,5.0,charles mckinney,the only thing I dont like is crack heads and ...,Hamburger Restaurant; Fast Food Restaurant,"Sep 9, 2011",1.315601e+09,2011-09-09 22:46:19,115962368504656773973,118446742455312620560
11453823,3.0,charles mckinney,Counter people are very impatient.,Sandwich Shop; Fast Food Restaurant; Caterer,"Mar 30, 2013",1.364705e+09,2013-03-31 06:49:00,116097431140849606669,118446742455312620560
11453827,2.0,charles mckinney,Too expensive for a fast food burger.,Hamburger Restaurant; Fast Food Restaurant,"Mar 30, 2013",1.364706e+09,2013-03-31 07:05:51,116458473784504954830,118446742455312620560


In [21]:
unique_ids = df_reviews['gPlusPlaceId'].unique()
df_places = df_places[df_places['gPlusPlaceId'].isin(unique_ids)]
display(df_places)

,name,price,phone,closed,gPlusPlaceId,address_line1,address_line2,address_line3,latitude,longitude,hours_monday,hours_tuesday,hours_wednesday,hours_thursday,hours_friday,hours_saturday,hours_sunday
3,T C's Referee Sports Bar,$$,(605) 361-2208,False,100327153115986850675,5322 W 26th St,"Sioux Falls, SD 57106",NaN,4.352949e+01,-9.679224e+01,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--10:00 pm
9,Old Chicago,$$,(503) 533-4650,False,118222137795476771294,17960 NW Evergreen Pkwy,"Beaverton, OR 97006",NaN,4.553518e+01,-1.228622e+02,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am
10,China Cottage,$$,(937) 294-4724,False,106432060150136868000,3718 Wilmington Pike,"Dayton, OH 45429",NaN,3.969290e+01,-8.413617e+01,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--10:00 pm,11:00 am--11:00 pm,11:00 am--11:00 pm,11:00 am--10:00 pm
23,Smokey Mountain Wings,$$,(865) 465-3121,False,100184392614713668281,3607 Outdoor Sportsman Pl,"Kodak, TN 37764",NaN,3.598598e+01,-8.361060e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,Sabatinos Italian Kitchen,$$$,(781) 646-4126,False,110300304875024740707,242 Massachusetts Ave,"Arlington, MA 02474",NaN,4.240690e+01,-7.114399e+01,10:00 am--10:15 pm,10:00 am--10:15 pm,10:00 am--10:15 pm,10:00 am--10:15 pm,10:00 am--10:30 pm,10:00 am--10:30 pm,10:00 am--9:45 pm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114315,Coast Coffee Cafe,$$$,(08) 9481 8868,False,102445991423518108977,1269 Hay St,West Perth WA 6005,Australia,-3.194821e+01,1.158403e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3114320,Alvernon Donut Shop,$$$,(520) 326-3307,False,105165247399326889634,1450 S Alvernon Way,"Tucson, AZ 85711",NaN,3.220395e+01,-1.109099e+02,5:00 am--5:00 pm,5:00 am--5:00 pm,5:00 am--5:00 pm,5:00 am--5:00 pm,5:00 am--5:00 pm,5:00 am--5:00 pm,5:00 am--5:00 pm
3114323,SUBWAY® Restaurants,$$$,(906) 265-7827,False,102934499497015360471,Riverside Ave,"Iron River, MI 49935",NaN,4.608880e+07,-8.863522e+07,8:00 am--9:00 pm,8:00 am--9:00 pm,8:00 am--9:00 pm,8:00 am--9:00 pm,8:00 am--9:00 pm,8:00 am--8:00 pm,9:00 am--8:00 pm
3114324,Super Stop & Shop,$$,(860) 404-0341,False,109070835041469072310,1799 Farmington Ave,"Unionville, CT 06085",NaN,4.175697e+01,-7.288338e+01,7:00 am--12:00 am,7:00 am--12:00 am,7:00 am--12:00 am,7:00 am--12:00 am,7:00 am--12:00 am,7:00 am--12:00 am,7:00 am--10:00 pm


#### Only including users who have reviewed

In [31]:
df_users.head()

,userName,currentPlace,education,gPlusUserId,jobs,previousPlaces
0,an lam,"['Thành phố Hồ Chí Minh, Việt Nam', [[], 10823...","[[[], [], [], [], [], 6], [['Đại học Kiến trúc...",100000010817154263736,"[['Tổng công ty IDICO', 'Chuyên viên Kỹ thuật ...","[['tp. Nam Định, Nam Định, Việt Nam', [[], 204..."
1,HALİL TURGUT,"['Adana', [[], 370000000, 353213330, 1]]","[[[], [], [], [], [], 6], [['Anadolu Universit...",100000013500285534661,"[['Mustafa Nevzat İlaç San. A.Ş.', 'Rebrezant'...",NaN
2,森田さとこ,NaN,"[[[], [], [], [], [], 6], []]",100000021336848867366,NaN,NaN
3,amey kore,NaN,"[[[], [], [], [], [], 6], []]",100000030557048145331,NaN,NaN
4,william spindler,NaN,"[[[], [], [], [], [], 6], []]",100000032416892623125,NaN,NaN


In [32]:
users = df_reviews['reviewerName']
df_users = df_users[df_users['userName'].isin(users)]
display(df_users)

,userName,currentPlace,education,gPlusUserId,jobs,previousPlaces
0,an lam,"['Thành phố Hồ Chí Minh, Việt Nam', [[], 10823...","[[[], [], [], [], [], 6], [['Đại học Kiến trúc...",100000010817154263736,"[['Tổng công ty IDICO', 'Chuyên viên Kỹ thuật ...","[['tp. Nam Định, Nam Định, Việt Nam', [[], 204..."
1,HALİL TURGUT,"['Adana', [[], 370000000, 353213330, 1]]","[[[], [], [], [], [], 6], [['Anadolu Universit...",100000013500285534661,"[['Mustafa Nevzat İlaç San. A.Ş.', 'Rebrezant'...",NaN
2,森田さとこ,NaN,"[[[], [], [], [], [], 6], []]",100000021336848867366,NaN,NaN
3,amey kore,NaN,"[[[], [], [], [], [], 6], []]",100000030557048145331,NaN,NaN
4,william spindler,NaN,"[[[], [], [], [], [], 6], []]",100000032416892623125,NaN,NaN
...,...,...,...,...,...,...
3747932,Andrei Mustea,NaN,"[[[], [], [], [], [], 6], []]",114675878141148336835,NaN,NaN
3747933,ahmed alalam,NaN,"[[[], [], [], [], [], 6], []]",114675879178455853299,NaN,NaN
3747934,Brad Shaffer,NaN,"[[[], [], [], [], [], 6], []]",114675884186004222559,NaN,NaN
3747935,Ashley Krueger,NaN,"[[[], [], [], [], [], 6], []]",114675887865682990096,NaN,NaN


In [34]:
df_reviews.columns

Index(['rating', 'reviewerName', 'reviewText', 'categories', 'reviewTime',
       'unixReviewTime', 'formattedDate', 'gPlusPlaceId', 'gPlusUserId'],
      dtype='object')

### Save all the cleaned datasets

In [33]:
df_users.to_csv('../data/users.csv', index=False)
df_places.to_csv('../data/places.csv', index=False)
df_reviews.to_csv('../data/reviews.csv', index=False)

In [3]:
df_sen = pd.read_csv(data_path + "review_sentiments.csv")


FileNotFoundError: [Errno 2] No such file or directory: '../data/review_sentiment.csv'